In [1]:
import pandas as pd
from git_root import git_root
import sqlite3

# where do the gold and ncbi portions of this SQLite database come from?
# previously

# also get reverse ? envo/mixs proportial mappings to gold

In [2]:
# replicates some of gold-path-mixs-triad-counts.ipynb

dbFile = git_root("data/gold-biosample-subset.db")
conn = sqlite3.connect(dbFile)
sql = """
select distinct 
  ECOSYSTEM
  , ECOSYSTEM_CATEGORY
  , ECOSYSTEM_TYPE
  , ECOSYSTEM_SUBTYPE
  , SPECIFIC_ECOSYSTEM
  , BROAD_SCALE_LABEL
  , LOCAL_SCALE_LABEL
  , MEDIUM_LABEL
  , ENV_BROAD_SCALE
  , ENV_LOCAL_SCALE
  , ENV_MEDIUM
  , count(*) as COUNT
from biosample
group by
    ECOSYSTEM
  , ECOSYSTEM_CATEGORY
  , ECOSYSTEM_TYPE
  , ECOSYSTEM_SUBTYPE
  , SPECIFIC_ECOSYSTEM
  , BROAD_SCALE_LABEL
  , LOCAL_SCALE_LABEL
  , MEDIUM_LABEL

order by count(*) desc
"""

gp_mixs_combo_counts = pd.read_sql_query(sql, conn)

gp_mixs_combo_counts.head()

,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,SPECIFIC_ECOSYSTEM,BROAD_SCALE_LABEL,LOCAL_SCALE_LABEL,MEDIUM_LABEL,ENV_BROAD_SCALE,ENV_LOCAL_SCALE,ENV_MEDIUM,COUNT
0,Environmental,Aquatic,Marine,Unclassified,Unclassified,marine biome,marine water body,sea water,ENVO_00000447,ENVO_00001999,ENVO_00002149,2754
1,Environmental,Aquatic,Marine,Oceanic,Unclassified,marine biome,marine water body,sea water,ENVO_00000447,ENVO_00001999,ENVO_00002149,1496
2,Environmental,Air,Outdoor Air,Unclassified,Unclassified,,atmospheric layer,air,,ENVO_01000543,ENVO_00002005,1117
3,Environmental,Aquatic,Marine,Inlet,Unclassified,marine biome,coastal inlet,sea water,ENVO_00000447,ENVO_00000137,ENVO_00002149,1104
4,Environmental,Aquatic,Marine,Epipelagic,Unclassified,oceanic epipelagic zone biome,,sea water,ENVO_01000035,,ENVO_00002149,1016


In [3]:
gp_mixs_combo_counts["gold_path_cat"] = (
    gp_mixs_combo_counts.ECOSYSTEM
    + "_"
    + gp_mixs_combo_counts.ECOSYSTEM_CATEGORY
    + "_"
    + gp_mixs_combo_counts.ECOSYSTEM_TYPE
    + "_"
    + gp_mixs_combo_counts.ECOSYSTEM_SUBTYPE
    + "_"
    + gp_mixs_combo_counts.SPECIFIC_ECOSYSTEM
)

gp_mixs_combo_counts["mixs_cat"] = (
    gp_mixs_combo_counts.BROAD_SCALE_LABEL
    + "_"
    + gp_mixs_combo_counts.LOCAL_SCALE_LABEL
    + "_"
    + gp_mixs_combo_counts.MEDIUM_LABEL
)

In [4]:
gp_mixs_combo_counts.COUNT.fillna(0, inplace=True)
gp_mixs_combo_counts.fillna("-", inplace=True)
gp_mixs_combo_counts = gp_mixs_combo_counts.astype(str)
gp_mixs_combo_counts["COUNT"] = gp_mixs_combo_counts["COUNT"].astype(float)

count_max = gp_mixs_combo_counts["COUNT"].max()

gp_mixs_combo_counts["count_prop"] = gp_mixs_combo_counts["COUNT"] / count_max

In [5]:
gp_mixs_combo_counts_mincols = gp_mixs_combo_counts[
    ["gold_path_cat", "mixs_cat", "COUNT"]
]

gp_mixs_combo_counts_mincols

,gold_path_cat,mixs_cat,COUNT
0,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_marine water body_sea water,2754.0
1,Environmental_Aquatic_Marine_Oceanic_Unclassified,marine biome_marine water body_sea water,1496.0
2,Environmental_Air_Outdoor Air_Unclassified_Unc...,_atmospheric layer_air,1117.0
3,Environmental_Aquatic_Marine_Inlet_Unclassified,marine biome_coastal inlet_sea water,1104.0
4,Environmental_Aquatic_Marine_Epipelagic_Unclas...,oceanic epipelagic zone biome__sea water,1016.0
...,...,...,...
1180,Environmental_Terrestrial_Volcanic_Fumaroles_U...,terrestrial biome_volcano_sediment,1.0
1181,Environmental_Terrestrial_Volcanic_Unclassifie...,terrestrial biome_mud volcano_sediment,1.0
1182,Environmental_Unclassified_Unclassified_Unclas...,__,1.0
1183,Environmental_Unclassified_Unclassified_Unclas...,terrestrial biome_fen_surface soil,1.0


In [6]:
summed_gp_counts = gp_mixs_combo_counts_mincols.groupby(["gold_path_cat"]).sum()
summed_gp_counts.columns = ["sum"]
summed_gp_counts.sort_values("sum", ascending=False)

,sum
gold_path_cat,
Environmental_Aquatic_Marine_Unclassified_Unclassified,3435.0
Environmental_Terrestrial_Soil_Unclassified_Unclassified,2531.0
Environmental_Aquatic_Marine_Oceanic_Unclassified,1902.0
Environmental_Terrestrial_Soil_Unclassified_Forest Soil,1549.0
Environmental_Aquatic_Freshwater_Lake_Unclassified,1493.0
...,...
Environmental_Aquatic_Non-marine Saline and Alkaline_Near-boiling (>90C)_Alkaline,1.0
Environmental_Aquatic_Non-marine Saline and Alkaline_Saline_Thalassic,1.0
Environmental_Aquatic_Thermal springs_Tepid (25-34C)_Sediment,1.0


In [7]:
gp_explained_by_mixs = gp_mixs_combo_counts_mincols.merge(
    summed_gp_counts, left_on="gold_path_cat", right_index=True
)
gp_explained_by_mixs["proportion"] = (
    gp_explained_by_mixs["COUNT"] / gp_explained_by_mixs["sum"]
)
gp_explained_by_mixs.drop("COUNT", axis=1, inplace=True)
gp_explained_by_mixs

,gold_path_cat,mixs_cat,sum,proportion
0,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_marine water body_sea water,3435.0,0.801747
17,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_deep chlorophyll maximum layer_se...,3435.0,0.074236
89,Environmental_Aquatic_Marine_Unclassified_Uncl...,oceanic mesopelagic zone biome_marine mesopela...,3435.0,0.020961
130,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_marine water body_surface water,3435.0,0.014265
148,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_marine water body_planktonic mate...,3435.0,0.011936
...,...,...,...,...
1143,Environmental_Terrestrial_Soil_Unclassified_Sh...,terrestrial biome_rhizosphere_soil,1.0,1.000000
1181,Environmental_Terrestrial_Volcanic_Unclassifie...,terrestrial biome_mud volcano_sediment,1.0,1.000000
1182,Environmental_Unclassified_Unclassified_Unclas...,__,3.0,0.333333
1183,Environmental_Unclassified_Unclassified_Unclas...,terrestrial biome_fen_surface soil,3.0,0.333333


In [8]:
summed_mixs_counts = gp_mixs_combo_counts_mincols.groupby(["mixs_cat"]).sum()
summed_mixs_counts.columns = ["sum"]
summed_mixs_counts.sort_values("sum", ascending=False)

,sum
mixs_cat,
marine biome_marine water body_sea water,4307.0
marine biome_coastal inlet_sea water,1292.0
__,1201.0
_atmospheric layer_air,1117.0
oceanic epipelagic zone biome__sea water,1016.0
...,...
aquatic biome_lake_biofilm,1.0
__air,1.0
terrestrial biome_fen_surface soil,1.0


In [9]:
mixs_explained_by_gp = gp_mixs_combo_counts_mincols.merge(
    summed_mixs_counts, left_on="mixs_cat", right_index=True
)
mixs_explained_by_gp["proportion"] = (
    mixs_explained_by_gp["COUNT"] / mixs_explained_by_gp["sum"]
)
mixs_explained_by_gp.drop("COUNT", axis=1, inplace=True)
mixs_explained_by_gp

,gold_path_cat,mixs_cat,sum,proportion
0,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_marine water body_sea water,4307.0,0.639424
1,Environmental_Aquatic_Marine_Oceanic_Unclassified,marine biome_marine water body_sea water,4307.0,0.347342
275,Environmental_Aquatic_Marine_Oceanic_Photic zone,marine biome_marine water body_sea water,4307.0,0.003947
296,Environmental_Aquatic_Marine_Gulf_Unclassified,marine biome_marine water body_sea water,4307.0,0.003483
312,Environmental_Aquatic_Unclassified_Unclassifie...,marine biome_marine water body_sea water,4307.0,0.003251
...,...,...,...,...
1175,Environmental_Terrestrial_Unclassified_Unclass...,polar biome_cryoconite hole_ice,1.0,1.000000
1179,Environmental_Terrestrial_Unclassified_Unclass...,terrestrial biome_stalagmite_rock,1.0,1.000000
1180,Environmental_Terrestrial_Volcanic_Fumaroles_U...,terrestrial biome_volcano_sediment,1.0,1.000000
1181,Environmental_Terrestrial_Volcanic_Unclassifie...,terrestrial biome_mud volcano_sediment,1.0,1.000000


In [10]:
gp_mixs_proportional_explanation = gp_mixs_combo_counts.merge(
    gp_explained_by_mixs,
    left_on=["gold_path_cat", "mixs_cat"],
    right_on=["gold_path_cat", "mixs_cat"],
)


gp_mixs_proportional_explanation = gp_mixs_proportional_explanation.merge(
    mixs_explained_by_gp,
    left_on=["gold_path_cat", "mixs_cat"],
    right_on=["gold_path_cat", "mixs_cat"],
    suffixes=("_gp", "_mixs"),
)

## `interest` column is intended to highlight Gold/MIxS combinations 
that are common and proportionately enriched.

A Chi-squared test might be more reliable?

The "high interest" combinations frequently contain some "Unclassified" or blank annotations.

In [11]:
gp_mixs_proportional_explanation["interest"] = (
    gp_mixs_proportional_explanation["count_prop"]
    * gp_mixs_proportional_explanation["proportion_gp"]
    * gp_mixs_proportional_explanation["proportion_mixs"]
)

gp_mixs_proportional_explanation = gp_mixs_proportional_explanation.sort_values(
    "interest", ascending=False
)

gp_mixs_proportional_explanation

,ECOSYSTEM,ECOSYSTEM_CATEGORY,ECOSYSTEM_TYPE,ECOSYSTEM_SUBTYPE,SPECIFIC_ECOSYSTEM,BROAD_SCALE_LABEL,LOCAL_SCALE_LABEL,MEDIUM_LABEL,ENV_BROAD_SCALE,ENV_LOCAL_SCALE,ENV_MEDIUM,COUNT,gold_path_cat,mixs_cat,count_prop,sum_gp,proportion_gp,sum_mixs,proportion_mixs,interest
0,Environmental,Aquatic,Marine,Unclassified,Unclassified,marine biome,marine water body,sea water,ENVO_00000447,ENVO_00001999,ENVO_00002149,2754.0,Environmental_Aquatic_Marine_Unclassified_Uncl...,marine biome_marine water body_sea water,1.000000,3435.0,0.801747,4307.0,0.639424,5.126563e-01
2,Environmental,Air,Outdoor Air,Unclassified,Unclassified,,atmospheric layer,air,,ENVO_01000543,ENVO_00002005,1117.0,Environmental_Air_Outdoor Air_Unclassified_Unc...,_atmospheric layer_air,0.405592,1147.0,0.973845,1117.0,1.000000,3.949835e-01
4,Environmental,Aquatic,Marine,Epipelagic,Unclassified,oceanic epipelagic zone biome,,sea water,ENVO_01000035,,ENVO_00002149,1016.0,Environmental_Aquatic_Marine_Epipelagic_Unclas...,oceanic epipelagic zone biome__sea water,0.368918,1016.0,1.000000,1016.0,1.000000,3.689179e-01
3,Environmental,Aquatic,Marine,Inlet,Unclassified,marine biome,coastal inlet,sea water,ENVO_00000447,ENVO_00000137,ENVO_00002149,1104.0,Environmental_Aquatic_Marine_Inlet_Unclassified,marine biome_coastal inlet_sea water,0.400871,1122.0,0.983957,1292.0,0.854489,3.370450e-01
6,Environmental,Aquatic,Marine,Coastal,Unclassified,marine biome,coastal water body,coastal sea water,ENVO_00000447,ENVO_02000049,ENVO_00002150,677.0,Environmental_Aquatic_Marine_Coastal_Unclassified,marine biome_coastal water body_coastal sea water,0.245824,877.0,0.771950,700.0,0.967143,1.835289e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,Environmental,Aquatic,Marine,Oceanic,Aphotic zone,marine biome,marine water body,sea water,ENVO_00000447,ENVO_00001999,ENVO_00002149,1.0,Environmental_Aquatic_Marine_Oceanic_Aphotic zone,marine biome_marine water body_sea water,0.000363,63.0,0.015873,4307.0,0.000232,1.338199e-09
1152,Environmental,Terrestrial,Soil,Unclassified,Unclassified,terrestrial biome,archeological site,fossil,ENVO_00000446,ENVO_00000564,ENVO_00002164,1.0,Environmental_Terrestrial_Soil_Unclassified_Un...,terrestrial biome_archeological site_fossil,0.000363,2531.0,0.000395,112.0,0.008929,1.280931e-09
762,Environmental,Aquatic,Marine,Coastal,Unclassified,marine biome,marine water body,sea water,ENVO_00000447,ENVO_00001999,ENVO_00002149,2.0,Environmental_Aquatic_Marine_Coastal_Unclassified,marine biome_marine water body_sea water,0.000726,877.0,0.002281,4307.0,0.000464,7.690447e-10
1128,Environmental,Terrestrial,Soil,Sand,Unclassified,marine biome,marine water body,sea water,ENVO_00000447,ENVO_00001999,ENVO_00002149,1.0,Environmental_Terrestrial_Soil_Sand_Unclassified,marine biome_marine water body_sea water,0.000363,124.0,0.008065,4307.0,0.000232,6.798913e-10


In [12]:
gp_mixs_proportional_explanation.to_csv(
    "gp_mixs_proportional_explanation.tsv", sep="\t", index=False
)